In [101]:
import torch 
from torch import nn

In [102]:
def corrd2( x , k ):
    h , w = k.shape
    y = torch.zeros( x.shape[0]-h+1 , x.shape[1]-w+1 )
    for i in range( y.shape[0] ):
        for j in range( y.shape[1] ):
            y[i,j] = ( x[ i : i+h , j : j+w ] * k).sum()
    return y

In [103]:
x = torch.arange(16).reshape( 4,4)
k = torch.tensor( [[1,0],[0,1] ] ,dtype=torch.float32, requires_grad=True )
x,k,corrd2( x,k)

(tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]]),
 tensor([[1., 0.],
         [0., 1.]], requires_grad=True),
 tensor([[ 5.,  7.,  9.],
         [13., 15., 17.],
         [21., 23., 25.]], grad_fn=<CopySlices>))

## 卷积层

In [104]:
class Cov2d_test( nn.Module ):
    def __init__( self  , *kernel_size ):
        super().__init__()
        self.weight = nn.Parameter( torch.ones( kernel_size ) ,requires_grad= True)
        # self.bias = nn.Parameter( torch.zeros( 1 ) , requires_grad= True) #在神经网络中卷积是在求和，所以只需要一个偏置
    def forward( self , x ):
        return corrd2( x , self.weight ) 

In [105]:
conv1 = Cov2d_test( 2,2 )
x = torch.arange( 8).reshape( 2 , 4 )
x , conv1( x ) , conv1.weight.data,conv1.weight.grad

(tensor([[0, 1, 2, 3],
         [4, 5, 6, 7]]),
 tensor([[10., 14., 18.]], grad_fn=<CopySlices>),
 tensor([[1., 1.],
         [1., 1.]]),
 None)

# 卷积迭代实验

In [106]:
class Cov2d_test( nn.Module ):
    def __init__( self  , *kernel_size ):
        super().__init__()
        self.weight = nn.Parameter( torch.ones( kernel_size ) ,requires_grad= True)
        # self.bias = nn.Parameter( torch.zeros( 1 ) , requires_grad= True) #在神经网络中卷积是在求和，所以只需要一个偏置
    def forward( self , x ):
        return corrd2( x , self.weight ) 
cov1 = Cov2d_test( 2,2 )
cov1.weight

Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [107]:
x = torch.arange(8).reshape( 2,4 )
y = corrd2(x , torch.Tensor( [[1,0],[0,1] ]))
x,y

(tensor([[0, 1, 2, 3],
         [4, 5, 6, 7]]),
 tensor([[5., 7., 9.]]))

In [108]:
for i in range(200):
    y_hat = cov1(x)
    l = (y_hat-y)**2
    cov1.zero_grad()
    l.sum().backward()
    cov1.weight.data -= 0.001*cov1.weight.grad
    if (i+1)%10 ==0:
        print(f'batch:{i} , loss:{l.sum()}')

batch:9 , loss:0.4637046456336975
batch:19 , loss:0.41661059856414795
batch:29 , loss:0.3845173120498657
batch:39 , loss:0.35489803552627563
batch:49 , loss:0.3275596797466278
batch:59 , loss:0.30232682824134827
batch:69 , loss:0.27903860807418823
batch:79 , loss:0.2575428783893585
batch:89 , loss:0.23770400881767273
batch:99 , loss:0.21939361095428467
batch:109 , loss:0.20249325037002563
batch:119 , loss:0.1868947148323059
batch:129 , loss:0.17249828577041626
batch:139 , loss:0.1592099815607071
batch:149 , loss:0.14694596827030182
batch:159 , loss:0.13562601804733276
batch:169 , loss:0.12517939507961273
batch:179 , loss:0.115536168217659
batch:189 , loss:0.10663630068302155
batch:199 , loss:0.09842219948768616


In [109]:
pre_y =corrd2(x , cov1.weight)
pre_y,cov1.weight#显然我们当时设置的权值是[[1,0],[0,1]，而这里的权值是[[0.6710, 0.6244],[0.4845, 0.4378]]，显然过拟合了。

(tensor([[4.7513, 6.9689, 9.1866]], grad_fn=<CopySlices>),
 Parameter containing:
 tensor([[0.6710, 0.6244],
         [0.4845, 0.4378]], requires_grad=True))

## 卷积

In [110]:
x = torch.ones( (6,8))
x[:,2:6]=0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [111]:
k = torch.tensor( [[1. , -1. ]])
y = corrd2( x,k)
y,y.shape

(tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]),
 torch.Size([6, 7]))

In [112]:
cov2 = nn.Conv2d( 1,1,kernel_size=(1,2) , bias=False )
x = x.reshape(1,1,6,8)
y = y.reshape( (1,1,6,7))
print( cov2.weight.grad )
for i in range( 10 ):
    y_hat = cov2( x )
    l = (y_hat-y)**2
    cov2.zero_grad()
    l.sum().backward()
    cov2.weight.data[:] -= 0.01*cov2.weight.grad
    print( f'batch:{i} , loss:{l.sum()}')

None
batch:0 , loss:25.960784912109375
batch:1 , loss:20.09854507446289
batch:2 , loss:15.563437461853027
batch:3 , loss:12.052184104919434
batch:4 , loss:9.333189010620117
batch:5 , loss:7.227618217468262
batch:6 , loss:5.597066879272461
batch:7 , loss:4.334368705749512
batch:8 , loss:3.3565356731414795
batch:9 , loss:2.5993010997772217


In [113]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = x.reshape((1, 1, 6, 8))
Y = y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')


batch 2, loss 7.154
batch 4, loss 1.467
batch 6, loss 0.355
batch 8, loss 0.104
batch 10, loss 0.036
